# Lesson 8: Cross-validation

So far, we've learned about splitting our data into training and testing sets to validate our models. This helps ensure that the model we create on one sample performs well on another sample we want to predict. 

However, we don't have to use just TWO samples to train and test our models. Instead, we can split our data up into MULTIPLE samples to train and test on multiple segments of the data. This is called CROSS-VALIDATION. This allows us to ensure that our model predicts outcomes over a wider range of circumstances. 

Let's begin by importing our packages.

In [1]:
! conda install geopandas -qy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold



# All requested packages already installed.
# packages in environment at /opt/conda:
#
geopandas                 0.3.0                    py36_0    conda-forge


In [2]:
import os
os.getcwd()
os.chdir('/home/jovyan/assignment-08-cross-validation-drewgobbi')


Today we'll be looking at 311 service requests for rodent inspection and abatement aggregated at the Census block level. The data set is already prepared for you and available in the same folder as this assignment. Census blocks are a good geographic level to analyze rodent infestations because they are drawn along natural and human-made boundaries, like rivers and roads, that rats tend not to cross. 

We will look at the 'activity' variable, which indicates whether inspectors found rat burrows during an inspection (1) or not (0). Here we are looking only at inpsections in 2016. About 43 percent on inspections in 2016 led to inspectors finding and treating rat burrows, as you can see below.

In [3]:
data = pd.read_csv('rat_data_2016.csv')

In [4]:
data.columns

Index(['activity', 'alley_condition', 'bbl_hotel', 'bbl_multifamily_rental',
       'bbl_restaurant', 'bbl_single_family_rental', 'bbl_storage',
       'bbl_two_family_rental', 'communitygarden_area', 'communitygarden_id',
       'dcrapermit_addition', 'dcrapermit_demolition', 'dcrapermit_excavation',
       'dcrapermit_new_building', 'dcrapermit_raze', 'impervious_area',
       'month', 'num_mixed_use', 'num_non_residential', 'num_residential',
       'park', 'pct_mixed_use', 'pct_non_residential', 'pct_residential',
       'pop_density', 'sidewalk_grates', 'ssl_cndtn_Average_comm',
       'ssl_cndtn_Average_res', 'ssl_cndtn_Excellent_comm',
       'ssl_cndtn_Excellent_res', 'ssl_cndtn_Fair_comm', 'ssl_cndtn_Fair_res',
       'ssl_cndtn_Good_comm', 'ssl_cndtn_Good_res', 'ssl_cndtn_Poor_comm',
       'ssl_cndtn_Poor_res', 'ssl_cndtn_VeryGood_comm',
       'ssl_cndtn_VeryGood_res', 'tot_pop', 'well_activity', 'WARD'],
      dtype='object')

In [5]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
activity,2606.0,0.431696,0.495408,0.000000,0.000000,0.000000,1.000000,1.000000
alley_condition,2606.0,11.111282,8.900166,0.000000,4.000000,10.000000,16.000000,79.000000
bbl_hotel,2606.0,0.082118,0.376073,0.000000,0.000000,0.000000,0.000000,8.000000
bbl_multifamily_rental,2606.0,1.388718,2.376244,0.000000,0.000000,0.000000,2.000000,22.000000
bbl_restaurant,2606.0,0.569455,1.518526,0.000000,0.000000,0.000000,0.000000,17.000000
bbl_single_family_rental,2606.0,4.709133,8.375165,0.000000,1.000000,2.000000,5.000000,147.000000
bbl_storage,2606.0,0.002686,0.051768,0.000000,0.000000,0.000000,0.000000,1.000000
bbl_two_family_rental,2606.0,0.743668,1.378860,0.000000,0.000000,0.000000,1.000000,15.000000
communitygarden_area,2606.0,18.727920,326.332382,0.000000,0.000000,0.000000,0.000000,11004.319881
communitygarden_id,2606.0,0.242134,3.234069,0.000000,0.000000,0.000000,0.000000,80.000000


Recall from last week that, when we do predictive analysis, we usually are not interested in the relationship between two different variables as we are when we do traditional hypothesis testing. Instead, we're interested in training a model that generates predictions that best fit our target population. Therefore, when we are doing any kind of validation, including cross-validation, it is important for us to choose the metric by which we will evaluate the performance of our models. 

For this model, we will predict the locations of requests for rodent inspection and abatement in the District of Columbia. When we select a validation metric, it's important for us to think about what we want to optimize. For example, do we want to make sure that our top predictions accurately identify places with rodent infestations, so we don't send our inspectors on a wild goose chase? Then we may to look at the models precision, or what proportion of its positive predictions turn out to be positive. Or do we want to make sure we don't miss any infestations? If so, we may want to look at recall, or the proportion of positive cases that are correctly categorized by the model. If we care a lot about how the model ranks our observations, then we may want to look at the area under the ROC curve, or ROC-AUC, while if we care more about how well the model fits the data, or its "calibration," we may want to look at Brier score or logarithmic loss (log-loss).

In the case of rodent inspections, we most likely want to make sure that we send our inspectors to places where they are most likely to find rats and to avoid sending them on wild goose [rat] chases. Therefore, we will optimize for precision, which we will call from the metrics library in scikit-learn. 

The metrics library in scikit-learn provides a number of different options. You should take some time to look at the different metrics that are available to you and consider which ones are most appropriate for your own research


In [6]:
from sklearn.metrics import precision_score

The next important decision we need to make when cross-validating our models is how we will define our "folds." Folds are the independent subsamples on which we train and test the data. Keep in mind that it is important that our folds are INDEPENDENT, which means we must guarantee that there's no overlap between our training and test set (i.e., no observation is in both the training and test set). Independence can also have other implications for how we slice the data, which we will discuss as we progress through this lesson.

One of the most common approaches to cross-validation is to make random splits in the data. This is often referred to as k-fold cross-validation, in which the only thing we define is the number of folds (k) that want to split our sample into. Here, I'll use the KFold function from scikit-learn's model_selection library. Let's begin by importing the library and then taking a look at how it splits our data.

In [7]:
from sklearn.model_selection import KFold

KFold divides our data into a pre-specified number of (approximately) equally-sized folds so that each observation is in the test set once. When we specify that shuffle=True, KFold first shuffles our data into a random order to ensure that the observations are randomly selected. By selecting a random_state, we can ensure that KFold selects observations the same way each time. 

While there are other functions in the model_selection library that will do much of this work for us, KFold will allow us to look at what's going on in the background of our cross-validation process. Let's begin by just looking at how KFold splits our data. Here we split our data into 10 folds each with 10 percent of the data (.1).

In [8]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)
for train_index, test_index in cv.split(data):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [   0    1    2 ..., 2603 2604 2605] TEST: [   9   22   27   33   53   70   92  104  109  117  121  135  137  156  182
  192  195  196  215  217  224  227  252  259  271  276  289  314  317  326
  333  351  398  399  418  422  427  436  438  443  452  465  478  480  482
  489  518  547  562  563  567  569  578  581  597  609  616  618  619  674
  682  686  700  704  710  711  720  722  728  743  745  746  748  764  778
  795  817  831  855  868  878  880  899  913  916  921  927  933  961  962
  982  983  988  998 1000 1012 1013 1018 1023 1032 1036 1051 1052 1059 1078
 1079 1096 1100 1101 1106 1108 1109 1147 1187 1192 1213 1263 1264 1285 1287
 1300 1323 1326 1327 1371 1396 1418 1421 1432 1452 1484 1507 1515 1520 1544
 1568 1570 1577 1580 1585 1588 1590 1592 1597 1622 1627 1656 1657 1668 1680
 1681 1686 1689 1708 1710 1719 1729 1732 1735 1757 1761 1762 1763 1765 1780
 1783 1790 1798 1803 1814 1816 1818 1820 1821 1827 1832 1836 1853 1873 1874
 1895 1898 1901 1902 1921 1927 1929 19

You can see that ShuffleSplit has selected a random set of observations from the index of our data set for each fold of our cross-validation. Let's look at the size of our training and test set for each fold.

In [9]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)
for train_index, test_index in cv.split(data):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))

TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2345 TEST: 261
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260
TRAIN: 2346 TEST: 260


Now let's try using KFold to train and test our model on 10 different subsets of our data. Below we set our cross-validator as 'cv'. We then loop through the various splits in our data that cv creates and use it to make our training and test sets. We then use our training set to fit a Logistic Regression model and generate predictions from our test set, which we compare to the actual outcomes we observed.

In [10]:
data.dtypes

activity                    float64
alley_condition             float64
bbl_hotel                   float64
bbl_multifamily_rental      float64
bbl_restaurant              float64
bbl_single_family_rental    float64
bbl_storage                 float64
bbl_two_family_rental       float64
communitygarden_area        float64
communitygarden_id          float64
dcrapermit_addition         float64
dcrapermit_demolition       float64
dcrapermit_excavation       float64
dcrapermit_new_building     float64
dcrapermit_raze             float64
impervious_area             float64
month                         int64
num_mixed_use               float64
num_non_residential         float64
num_residential             float64
park                        float64
pct_mixed_use               float64
pct_non_residential         float64
pct_residential             float64
pop_density                 float64
sidewalk_grates             float64
ssl_cndtn_Average_comm      float64
ssl_cndtn_Average_res       

In [11]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=0)

## Create for-loop
for train_index, test_index in cv.split(data):

    ## Define training and test sets
    X_train = data.loc[train_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_train = data.loc[train_index]['activity']
    X_test = data.loc[test_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_test = data.loc[test_index]['activity']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))

Precision: 56.9
Precision: 55.8
Precision: 60.4
Precision: 63.5
Precision: 55.4
Precision: 52.8
Precision: 56.9
Precision: 57.1
Precision: 65.8
Precision: 41.0


We can see that, for the most part, about 50 to 60 percent of the inspections our model predicts will lead our inspectors to rat burrows actually do. This is a modest improvement over our inspectors' current performance in the field. Based on these results, if we used our models to determine which locations our inspectors go to in the field, we'd probably see a 10 to 20 point increase in their likelihood of finding rat burrows.

## Exercise 1

Try running the k-fold cross-validation a few times with the same random state. Then try running it a few times with different random states. How do the results change?

In [12]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=0)

## Create for-loop
for train_index, test_index in cv.split(data):

    ## Define training and test sets
    X_train = data.loc[train_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_train = data.loc[train_index]['activity']
    X_test = data.loc[test_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_test = data.loc[test_index]['activity']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))

Precision: 56.9
Precision: 55.8
Precision: 60.4
Precision: 63.5
Precision: 55.4
Precision: 52.8
Precision: 56.9
Precision: 57.1
Precision: 65.8
Precision: 41.0


In [13]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=1)

## Create for-loop
for train_index, test_index in cv.split(data):

    ## Define training and test sets
    X_train = data.loc[train_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_train = data.loc[train_index]['activity']
    X_test = data.loc[test_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_test = data.loc[test_index]['activity']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))

Precision: 60.3
Precision: 52.7
Precision: 57.4
Precision: 60.4
Precision: 57.1
Precision: 47.9
Precision: 54.2
Precision: 43.2
Precision: 71.4
Precision: 53.3


In [14]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=17)

## Create for-loop
for train_index, test_index in cv.split(data):

    ## Define training and test sets
    X_train = data.loc[train_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_train = data.loc[train_index]['activity']
    X_test = data.loc[test_index].drop(['activity', 'month', 'WARD'], axis=1)
    y_test = data.loc[test_index]['activity']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))
    

Precision: 60.0
Precision: 62.2
Precision: 62.7
Precision: 47.7
Precision: 53.2
Precision: 55.7
Precision: 43.6
Precision: 47.5
Precision: 55.4
Precision: 60.4


### Different random states create different measures of precision/datapoints across the 10 subsamples. However, the mean still seems to be in the mid 50's,suggesting some level of consitency

It's important to point out here that, because we have TIME SERIES data, the same Census blocks may be appearing in our training AND our test sets. This is a challenge to ensuring that our training and test samples are INDEPENDENT. While Rodent Control does not inspect the same blocks every month, some of the same blocks may be re-inspected from month to month depending on where 311 requests are coming from. 

However, this also affords us an opportunity. More than likely, when we make predictions about which inspections will lead our inspectors to rat burrows, we are interested in predicting FUTURE inspections with observations from PAST inspections. In this case, cross-validating over time can be a very good way of looking at how well our models are performing. 

Cross-validating over time requires more than just splitting by month. Rather, we will use observations from each month as a test set and train our models on all PRIOR months. Which we do below.

## Cross-validation by Month

Let's begin by seeing what our cross-validation sets look like. Below, we loop through each of the sets to see which months end up in our training and test sets. You can see that as we move from month to month, we have more and more past observations in our training set.

In [15]:
months = np.sort(data.month.unique())

for month in range(2,13):
    test = data[data.month==month]
    train = data[(data.month < month)]

    print('Test Month: '+str(test.month.unique()), 'Training Months: '+str(train.month.unique()))

Test Month: [2] Training Months: [1]
Test Month: [3] Training Months: [1 2]
Test Month: [4] Training Months: [1 2 3]
Test Month: [5] Training Months: [1 2 3 4]
Test Month: [6] Training Months: [1 2 3 4 5]
Test Month: [7] Training Months: [1 2 3 4 5 6]
Test Month: [8] Training Months: [1 2 3 4 5 6 7]
Test Month: [9] Training Months: [1 2 3 4 5 6 7 8]
Test Month: [10] Training Months: [1 2 3 4 5 6 7 8 9]
Test Month: [11] Training Months: [ 1  2  3  4  5  6  7  8  9 10]
Test Month: [12] Training Months: [ 1  2  3  4  5  6  7  8  9 10 11]


In [16]:
months = np.sort(data.month.unique())

for month in range(2,13):

    test = data[data.month==month]
    train = data[(data.month < month)]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Month '+str(month)+': '+str(100*round(precision_score(y_test, predicted),3)))

Precision for Month 2: 79.2
Precision for Month 3: 67.9
Precision for Month 4: 48.9
Precision for Month 5: 63.5
Precision for Month 6: 61.3
Precision for Month 7: 67.9
Precision for Month 8: 67.2
Precision for Month 9: 67.6
Precision for Month 10: 57.3
Precision for Month 11: 68.3
Precision for Month 12: 70.5


Our model seems to be performing even better when we cross-validate over months, possibly because we're structuring the cross-validation such that inspections in some of the same blocks appear consistently over time. 

## Exercise 2

Try re-creating this cross-validation, but with the training set restricted to only the 3 months prior to the test set. Now do the same with the last 1 and 2 months. Do the results change?

In [17]:
months = np.sort(data.month.unique())

for month in range(2,13):
    test = data[data.month==month]
    train = data[(data.month > (month-4))]
    train = train[(train.month < month)]
                  
    print('Test Month: '+str(test.month.unique()), 'Training Months: '+str(train.month.unique()))

Test Month: [2] Training Months: [1]
Test Month: [3] Training Months: [1 2]
Test Month: [4] Training Months: [1 2 3]
Test Month: [5] Training Months: [2 3 4]
Test Month: [6] Training Months: [3 4 5]
Test Month: [7] Training Months: [4 5 6]
Test Month: [8] Training Months: [5 6 7]
Test Month: [9] Training Months: [6 7 8]
Test Month: [10] Training Months: [7 8 9]
Test Month: [11] Training Months: [ 8  9 10]
Test Month: [12] Training Months: [ 9 10 11]


In [18]:
## last three months 
months = np.sort(data.month.unique())

for month in range(2,13):
    test = data[data.month==month]
    train = data[(data.month > (month-4))]
    train = train[(train.month < month)]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Month '+str(month)+': '+str(100*round(precision_score(y_test, predicted),3)))
    


Precision for Month 2: 79.2
Precision for Month 3: 67.9
Precision for Month 4: 48.9
Precision for Month 5: 63.5
Precision for Month 6: 61.3
Precision for Month 7: 67.9
Precision for Month 8: 67.2
Precision for Month 9: 67.6
Precision for Month 10: 57.3
Precision for Month 11: 68.3
Precision for Month 12: 70.5


In [19]:
## last two months 
months = np.sort(data.month.unique())

for month in range(2,13):
    test = data[data.month==month]
    train = data[(data.month > (month-3))]
    train = train[(train.month < month)]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Month '+str(month)+': '+str(100*round(precision_score(y_test, predicted),3)))
    
    

Precision for Month 2: 79.2
Precision for Month 3: 67.9
Precision for Month 4: 48.9
Precision for Month 5: 63.5
Precision for Month 6: 61.3
Precision for Month 7: 67.9
Precision for Month 8: 67.2
Precision for Month 9: 67.6
Precision for Month 10: 57.3
Precision for Month 11: 68.3
Precision for Month 12: 70.5


In [20]:
## last two months 
months = np.sort(data.month.unique())

for month in range(2,13):
    test = data[data.month==month]
    train = data[(data.month==(month-1))]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Month '+str(month)+': '+str(100*round(precision_score(y_test, predicted),3)))
    
    

Precision for Month 2: 79.2
Precision for Month 3: 67.9
Precision for Month 4: 48.9
Precision for Month 5: 63.5
Precision for Month 6: 61.3
Precision for Month 7: 67.9
Precision for Month 8: 67.2
Precision for Month 9: 67.6
Precision for Month 10: 57.3
Precision for Month 11: 68.3
Precision for Month 12: 70.5


It's just as precise! 

We may still be concerned about the independence of our training and test sets. In particular, as I've pointed out, the same Census blocks may appear repeatedly in our data over time. In this case, it may be good to cross-validate geographically to make sure that our model is performing well in different parts of the city. In particular, we know that requests for rodent abatement (and rats themselves) are more common in some parts of the city than in others. In particular, rats are more common in the more densely-populated parts of downtown and less common in less densely-populated places like Wards 3, 7, and 8. Therefore, we may be interested in cross-validating by ward. 

Again, this is as simple as looping through each of the 8 wards, holding out each ward as a test set and training the models on observations from the remaining wards.

## Cross-validate by Ward

In [21]:
data.WARD.value_counts().sort_index()

1    419
2    477
3    105
4    496
5    355
6    458
7    154
8    142
Name: WARD, dtype: int64

In [22]:
for ward in np.sort(data.WARD.unique()):

    test = data[data.WARD == ward]
    train = data[data.WARD != ward]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Ward '+str(ward)+': '+str(100*round(precision_score(y_test, predicted),3)))

Precision for Ward 1: 60.5
Precision for Ward 2: 60.2
Precision for Ward 3: 81.2
Precision for Ward 4: 62.0
Precision for Ward 5: 62.3
Precision for Ward 6: 55.6
Precision for Ward 7: 0.0
Precision for Ward 8: 0.0


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Here we see that the model performs very well predicting the outcomes of inspections in wards 1 through 4, but less well in wards 5 though 8. In wards 7 and 8 in particular, the model fails to predict any positive cases. This means that our model may be overfit to observations in Wards 1 through 6, and we may want to re-evaluate our approach. 

## Exercise 3

Explore the data and our model and try to come up with some reasons that the model is performing poorly on Wards 7 and 8. Is there a way we can fix the model to perform better on those wards? How might we fix the model?

In [23]:
data.head().T

,0,1,2,3,4
activity,1.000000,0.000000,0.000000,1.000000,1.000000
alley_condition,10.000000,25.000000,15.000000,0.000000,10.000000
bbl_hotel,0.000000,0.000000,0.000000,0.000000,0.000000
bbl_multifamily_rental,1.000000,0.000000,0.000000,1.000000,2.000000
bbl_restaurant,1.000000,0.000000,0.000000,0.000000,0.000000
bbl_single_family_rental,2.000000,8.000000,3.000000,3.000000,1.000000
bbl_storage,0.000000,0.000000,0.000000,0.000000,0.000000
bbl_two_family_rental,0.000000,0.000000,0.000000,0.000000,0.000000
communitygarden_area,0.000000,0.000000,0.000000,0.000000,0.000000
communitygarden_id,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
activity,2606.0,0.431696,0.495408,0.000000,0.000000,0.000000,1.000000,1.000000
alley_condition,2606.0,11.111282,8.900166,0.000000,4.000000,10.000000,16.000000,79.000000
bbl_hotel,2606.0,0.082118,0.376073,0.000000,0.000000,0.000000,0.000000,8.000000
bbl_multifamily_rental,2606.0,1.388718,2.376244,0.000000,0.000000,0.000000,2.000000,22.000000
bbl_restaurant,2606.0,0.569455,1.518526,0.000000,0.000000,0.000000,0.000000,17.000000
bbl_single_family_rental,2606.0,4.709133,8.375165,0.000000,1.000000,2.000000,5.000000,147.000000
bbl_storage,2606.0,0.002686,0.051768,0.000000,0.000000,0.000000,0.000000,1.000000
bbl_two_family_rental,2606.0,0.743668,1.378860,0.000000,0.000000,0.000000,1.000000,15.000000
communitygarden_area,2606.0,18.727920,326.332382,0.000000,0.000000,0.000000,0.000000,11004.319881
communitygarden_id,2606.0,0.242134,3.234069,0.000000,0.000000,0.000000,0.000000,80.000000


In [25]:
data.bbl_restaurant.value_counts()

0.0     2015
1.0      260
2.0      137
3.0       83
4.0       38
5.0       24
8.0       11
6.0       10
9.0        9
7.0        7
12.0       4
10.0       2
15.0       2
11.0       2
17.0       1
14.0       1
Name: bbl_restaurant, dtype: int64

In [26]:
data.groupby('WARD').bbl_restaurant.value_counts()

WARD  bbl_restaurant
1     0.0               283
      1.0                61
      2.0                37
      3.0                18
      4.0                 8
      6.0                 2
      8.0                 2
      11.0                2
      12.0                2
      5.0                 1
      9.0                 1
      15.0                1
      17.0                1
2     0.0               255
      1.0                70
      2.0                49
      3.0                37
      4.0                23
      5.0                19
      6.0                 7
      7.0                 5
      9.0                 5
      8.0                 4
      10.0                2
      14.0                1
3     0.0                71
      1.0                11
      2.0                10
      3.0                 5
      9.0                 3
      7.0                 2
      4.0                 1
      5.0                 1
      8.0                 1
4     0.0               431

In [27]:
data.groupby(data.activity==1).WARD.value_counts().sort_values(ascending = False)

activity  WARD
False     6       353
True      2       271
False     4       260
True      4       236
          1       231
False     2       206
True      5       199
False     1       188
          5       156
          7       134
          8       119
True      6       105
False     3        65
True      3        40
          8        23
          7        20
Name: WARD, dtype: int64

In [28]:
data.groupby('WARD').tot_pop.sum().sort_values(ascending = False)

WARD
1    123400
2     90829
4     72237
6     66442
5     58077
3     27970
7     27699
8     25599
Name: tot_pop, dtype: int64

Looks like Ward 3, 7, 8 are about the same size. Model is MOST Accurate in Ward 3 and not accurate at all in Wards 7 and 8. Maybe our model is overfit to 3 -- what about these wards are different?

In [29]:
three = data[data.WARD==3]
seven = data[data.WARD==7]
eight = data[data.WARD==8]

three.activity.value_counts()

0.0    65
1.0    40
Name: activity, dtype: int64

In [30]:
seven.activity.value_counts()

0.0    134
1.0     20
Name: activity, dtype: int64

In [31]:
eight.activity.value_counts()

0.0    119
1.0     23
Name: activity, dtype: int64

In [32]:
data.groupby('WARD').activity.value_counts(sort=True)

WARD  activity
1     1.0         231
      0.0         188
2     1.0         271
      0.0         206
3     0.0          65
      1.0          40
4     0.0         260
      1.0         236
5     1.0         199
      0.0         156
6     0.0         353
      1.0         105
7     0.0         134
      1.0          20
8     0.0         119
      1.0          23
Name: activity, dtype: int64

Ward's 5-8 have different active-not active ratios. They're most out of whack in 6-8, the least accurate of our predictions. Ratio of not active/to active is about 1 (give or take .25) for every value. 

Things are about 3 times as inactive and 5-6 times as inactive in wards 6-8. Ward 3, the sample we could be overfitting to is about 1.5 times more inactive. A bit weird, but more reasonable. This could be a source of our issue?

In [33]:
###Per charlie's suggestion in slack, let's try the same regression with equal weights

for ward in np.sort(data.WARD.unique()):

    test = data[data.WARD == ward]
    train = data[data.WARD != ward]
    X_test = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_test = test['activity']
    X_train = test.drop(['activity', 'month', 'WARD'], axis=1)
    y_train = test['activity']
        
    clf = LogisticRegression(class_weight = 'balanced')
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print('Precision for Ward '+str(ward)+': '+str(100*round(precision_score(y_test, predicted),3)))


Precision for Ward 1: 66.3
Precision for Ward 2: 67.2
Precision for Ward 3: 71.8
Precision for Ward 4: 57.5
Precision for Ward 5: 69.4
Precision for Ward 6: 36.0
Precision for Ward 7: 25.0
Precision for Ward 8: 30.6


## Exercise 4

Now try running some cross-validations with the data from your project. What are some different ways you might slice the data you're using for your project? Try them out here. This will be a good way to begin making progress toward your final submission. 

PLEASE REMEMBER TO SUBMIT THIS HOMEWORK BY CLASS TIME ON THURSDAY.

### This was a lot tougher than anticipated...here's a brief write up of how we're planning to approach in the future

Our data set is HUGE and messy. To predict businesses licenses, we're going to need to devote a considereable amount of time cleaning and framing all of these variables. We have everything from economic tracts to tons of geodata in regards to certain landmarks (coffee shops, development zones, senior communities...everything) over 5 years throughout the entire district. Additionally, we will likely to have to take random samples of the data sets we are combining in order to get a more workable data set for our cloud environment to digest. 

These variables need to be standardized into floats and integers that sklearn can handles, merged, and framed accordingly. 

Afterwards, there are several interesting ways we can think about cross validating our model. In regards to geodata, it makes sense to evaluate how well our predictions at a ward to ward, zone, and potentially neighborhood level. 

It could also be interesting to validate across economic tracts/levels. Perhaps we under or overperform in high income areas, but our model is not so good at capturing growth in other segments. 

Finally, it's important that we validate our model over the time units in our project, whether that is years, weeks or months. This can give us a better idea of how accurate we are in the long and short term, and can help inform our objective. For example, a model that predicts businesses creation well year to year but not week to week or month to month may be just as relevant for city planning uses and other urban policy initiatives that are focused in longer term units. 

### Here is a really basic script I started to run, but had a lot of trouble getting through. The three of us will have to spend a lot of time framing our data before moving on to this step! 

In [34]:
bl_16 = pd.read_csv('https://opendata.arcgis.com/datasets/82ab09c9541b4eb8ba4b537e131998ce_22.csv',
                 encoding = 'utf-8', low_memory= False)
bl_15 = pd.read_csv('https://opendata.arcgis.com/datasets/4c4d6b4defdf4561b737a594b6f2b0dd_23.csv',
                 encoding = 'utf-8', low_memory= False)
bl_14 = pd.read_csv('https://opendata.arcgis.com/datasets/d7aa6d3a3fdc42c4b354b9e90da443b7_1.csv',
                 encoding = 'utf-8', low_memory= False)
bl_13 = pd.read_csv('https://opendata.arcgis.com/datasets/a8434614d90e416b80fbdfe2cb2901d8_2.csv',
                 encoding = 'utf-8', low_memory= False)
bl_12 = pd.read_csv('https://opendata.arcgis.com/datasets/c4368a66ce65455595a211d530facc54_3.csv',
                 encoding = 'utf-8', low_memory= False)
sets = [bl_12, bl_13, bl_14, bl_15, bl_16]
sets[1].columns
big = sets[0]
for i in range(1, 5):
    big = big.append(sets[i])
big.LICENSE_START_DATE = pd.to_datetime(big.LICENSE_START_DATE)
big.sort_values('LICENSE_START_DATE')
big['month'] = 0
big['month'] = big['LICENSE_START_DATE'].dt.year + (big['LICENSE_START_DATE'].dt.month/100)
big = big.dropna(subset=['month'])
big_group = big.set_index(['MARADDRESSREPOSITORYID','month'])
big_group = big_group.sort_index(ascending=True)
big_group.reset_index(inplace=True)
#big_group.month.value_counts(dropna=False)

In [35]:
big.head().T

,0,1,2,3,4
X,-77.033,-77.0125,-76.9946,-77.0274,-77.0055
Y,38.9329,38.8789,38.8774,38.8993,38.8772
OBJECTID,26213718,26210260,26210554,26214890,26215597
BBL_LICENSE_FACT_ID,314418,313394,311922,315270,316531
LICENSESTATUS,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
LICENSECATEGORY,Charitable Solicitation,Charitable Solicitation,Mechanical Amusement Machine,Charitable Solicitation,General Business Licenses
CUST_NUM,400213000107,400213000103,54001967,69001237,400313900559
TRADE_NAME,NEWGROUNDS FITNESS,JEREMIAH'S CIRCLE OF FRIENDS INC,FULLER BARBER SHOP,PRETRIAL JUSTICE INSTITUTE,"Washington Wags and Whiskers, LLC; Ashley E Ba..."
LICENSE_START_DATE,2012-12-01 00:00:00,2012-12-01 00:00:00,2010-06-01 00:00:00,2011-03-01 00:00:00,2012-12-01 00:00:00
LICENSE_EXPIRATION_DATE,2014-11-30T00:00:00.000Z,2014-11-30T00:00:00.000Z,2012-05-31T00:00:00.000Z,2013-02-28T00:00:00.000Z,2014-11-30T00:00:00.000Z


In [36]:
big.dtypes.sort_values()

WARD                                    int64
OBJECTID                                int64
BBL_LICENSE_FACT_ID                     int64
DCSTATLOCATIONKEY                       int64
DCSTATADDRESSKEY                        int64
CUST_NUM                                int64
MARADDRESSREPOSITORYID                  int64
X                                     float64
ZIPCODE                               float64
YCOORD                                float64
XCOORD                                float64
LATITUDE                              float64
LONGITUDE                             float64
month                                 float64
LICENSE_START_DATE             datetime64[ns]
Y                                     float64
HOTSPOT2004NAME                        object
HOTSPOT2005NAME                        object
HOTSPOT2006NAME                        object
NEIGHBORHOODCLUSTER                    object
PSA                                    object
DISTRICT                          

In [37]:
big1 = big.drop(['LICENSESTATUS','LICENSECATEGORY' , 'TRADE_NAME' , 'LICENSE_EXPIRATION_DATE', 
'LICENSE_ISSUE_DATE', 'AGENT_PHONE', 'LASTMODIFIEDDATE', 'CITY', 'STATE', 'SITEADDRESS',                         
'ANC', 'SMD', 'DISTRICT', 'PSA', 'NEIGHBORHOODCLUSTER','HOTSPOT2006NAME','HOTSPOT2005NAME', 'HOTSPOT2004NAME', 
'BUSINESSIMPROVEMENTDISTRICT'], axis =1)


In [38]:
big1.columns

Index(['X', 'Y', 'OBJECTID', 'BBL_LICENSE_FACT_ID', 'CUST_NUM',
       'LICENSE_START_DATE', 'LATITUDE', 'LONGITUDE', 'XCOORD', 'YCOORD',
       'ZIPCODE', 'MARADDRESSREPOSITORYID', 'DCSTATADDRESSKEY',
       'DCSTATLOCATIONKEY', 'WARD', 'month'],
      dtype='object')

In [52]:
big2 = big1.drop(['LICENSE_START_DATE','month'], axis =1)

In [54]:
big2.dtypes.sort_values()

OBJECTID                    int64
BBL_LICENSE_FACT_ID         int64
CUST_NUM                    int64
MARADDRESSREPOSITORYID      int64
DCSTATADDRESSKEY            int64
DCSTATLOCATIONKEY           int64
WARD                        int64
X                         float64
Y                         float64
LATITUDE                  float64
LONGITUDE                 float64
XCOORD                    float64
YCOORD                    float64
ZIPCODE                   float64
dtype: object

In [43]:
big2.BBL_LICENSE_FACT_ID = big2.BBL_LICENSE_FACT_ID.astype(float)
big2.OBJECTID = big2.OBJECTID.astype(float)
big2.CUST_NUM = big2.CUST_NUM.astype(float)
big2.

In [44]:
## Define function
cv = KFold(n_splits=10, shuffle=True, random_state=0)

## Create for-loop
for train_index, test_index in cv.split(big2):

    ## Define training and test sets
    X_train = big2.loc[train_index].drop(['BBL_LICENSE_FACT_ID'], axis=1)
    y_train = data.loc[train_index]['BBL_LICENSE_FACT_ID']
    X_test = data.loc[test_index].drop(['BBL_LICENSE_FACT_ID'], axis=1)
    y_test = data.loc[test_index]['BBL_LICENSE_FACT_ID']
        
    ## Fit model
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    ## Generate predictions
    predicted = clf.predict(X_test)
    
    ## Compare to actual outcomes and return precision
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))
    

KeyError: 'BBL_LICENSE_FACT_ID'